### Loading Libraries and Data

In [45]:
# Import necessary libraries
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import streamlit_jupyter as st

# Load datasets
suppliers = pd.read_excel("Data/MedicalEquipmentSuppliers.xls")
states = pd.read_csv("Data/states.csv")

In [2]:
missing_row_indices = states[states.isna().any(axis = 1)].index.tolist()
states.loc[missing_row_indices,]

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2016),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
6,Connecticut,9.1%,6%,-3.1%,110000,2148000,102917,80759,$357,True,NaN,761137,NaN,644136
19,Maine,10.1%,8.4%,-1.7%,22000,702000,75240,63896,$342,False,NaN,273160,NaN,315160
51,United States,15.5%,9.4%,6.1%,19304000,172292000,11081330,9389609,$291,NaN,56392477.0,73532931,16106157.0,57149984


In [3]:
split_suppliers = suppliers.copy(deep = True)
split_suppliers = split_suppliers.dropna(subset = ["specialitieslist", "supplieslist"]).reset_index(drop = True)
split_suppliers["specialitieslist"] = split_suppliers["specialitieslist"].str.split("|")
split_suppliers["supplieslist"] = split_suppliers["supplieslist"].str.split("|")

In [4]:
specialities = set()
for row in split_suppliers.itertuples():
    row_specialities = row.specialitieslist
    for supply in row_specialities:
        specialities.add(supply)

In [5]:
supplies = set()
for row in split_suppliers.itertuples():
    row_supplies = row.supplieslist
    for supply in row_supplies:
        supplies.add(supply)

In [9]:
# This code block was generated with assistance from ChatGPT (Version 5), accessed on 10/13/2025
mlb = MultiLabelBinarizer()
encoded_specialities = pd.DataFrame(
    mlb.fit_transform(split_suppliers["specialitieslist"]),
    columns = mlb.classes_,
    index = split_suppliers.index
)
encoded_supplies = pd.DataFrame(
    mlb.fit_transform(split_suppliers["supplieslist"]),
    columns = mlb.classes_,
    index = split_suppliers.index
)

suppliers_encoded = pd.concat([split_suppliers, encoded_specialities, encoded_supplies], axis = 1)
suppliers_encoded

,provider_id,acceptsassignement,participationbegindate,businessname,practicename,practiceaddress1,practiceaddress2,practicecity,practicestate,practicezip9code,...,Walkers,Wheelchair Seating/Cushions,Wheelchairs (Complex Rehab Manual Rel Accessories),Wheelchairs (Complex Rehab Power Rel Accessories),Wheelchairs (Complex Rehabilitative Manual),Wheelchairs (Complex Rehabilitative Power),Wheelchairs (Standard Manual Related Accessories),Wheelchairs (Standard Manual),Wheelchairs (Standard Power Related Accessories),Wheelchairs (Standard Power)
0,20615807,True,2025-07-07,SMITHS FOOD & DRUG CENTERS INC,SMITHS PHARMACY #354,3850 E FLAMINGO RD,NaN,LAS VEGAS,NV,891216227,...,1,0,0,0,0,0,0,0,0,0
1,20615153,True,2025-07-07,SMITHS FOOD & DRUG CENTERS INC,SMITHS PHARMACY #47,2039 E 9400 S,NaN,SANDY,UT,840933100,...,1,0,0,0,0,0,0,0,0,0
2,20615219,True,2025-07-07,SMITHS FOOD & DRUG CENTERS INC,FRY'S FOOD AND DRUG STORE #116,950 FAIR ST,NaN,PRESCOTT,AZ,863051806,...,1,0,0,0,0,0,0,0,0,0
3,20616047,True,2025-07-07,SMITHS FOOD & DRUG CENTERS INC,FRY'S FOOD AND DRUG #48,1385 E FLORENCE BLVD,NaN,CASA GRANDE,AZ,851225318,...,1,0,0,0,0,0,0,0,0,0
4,20615921,True,2025-07-07,SMITHS FOOD & DRUG CENTERS INC,FRYS FOOD AND DRUG #112,1575 N DYSART RD,NaN,AVONDALE,AZ,853231204,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58060,20679017,True,1984-10-01,UNIVERSITY OF TEXAS HEALTH SCIENCE CENTER AT S...,UTHSCSA REHAB ENGINEERING LAB,7703 FLOYD CURL DR,6TH FLOOR ROOM 638E,SAN ANTONIO,TX,78229,...,0,0,0,0,0,0,0,0,0,0
58061,20678369,True,1984-10-01,ALLINA HEALTH SYSTEM,ALLINA HEALTH CAMBRIDGE CLINIC,701 DELLWOOD ST S,NaN,CAMBRIDGE,MN,550081920,...,1,0,0,0,0,0,0,0,0,0
58062,20678603,True,1984-10-01,ALLINA HEALTH SYSTEM,ALLINA HEALTH HASTINGS CLINIC,1880 N FRONTAGE RD,NaN,HASTINGS,MN,550332687,...,0,0,0,0,0,0,0,0,0,0
58063,20677991,True,1984-10-01,ALLINA HEALTH SYSTEM,ALLINA HEALTH SYSTEM,14655 GALAXIE AVE,NaN,APPLE VALLEY,MN,551248575,...,0,0,0,0,0,0,0,0,0,0
